# **Introduction**
>*This Analysis of Paris Arrondissements, will include information pulled from the internet, including FourSquare Data to provide travelers, researchers, and other interested parties with neighborhood statistics such as density, population, landmarks, and locations.  Initially created to pull information on the 20 arrondissements of Paris, data on each arrondissement's 4 quarters seemed to be more relevant and detailed for exploration at the venue level.

# **Data Requirements and Descriptions**

### **Install and Import Libraries**

>*Setup the initial environment, using pip, it may be necessary to install the below models before importing modules </br>(some may not be needed, depending on prior usage and installations)*

>*For this project, I have choosen the following libraries and modules which will be used throughout the analysis*

In [873]:
!pip install geopandas
!pip install geopy
!pip install geocoderd
!pip install geoplot
!pip install folium
!pip install urllib3
import geopy
import pandas as pd
from geopy.geocoders import Bing
import numpy as np
import geopy
import geocoder
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import os 
import folium 

import json 
import requests

### 80 Parisian Neighborhoods or *'Quartiers'* within 20 Parisian *'Arrondissements'*.  


>As an initial starting point, we first scrape our list of neighborhoods from data within the website 'https://en.wikipedia.org/wiki/Quarters_of_Paris'.  
>
>This data will be the beginning of our neighborhood dataset.</br>
>
>*Use "requests.get" to grab the html which is then parsed with Pandas and stored in a list.*
>
>Once displayed we see 80 rows of data (4 quarters X 20 arrondissements), each with features detailing: </br>
>**Arrondissement Number** and 'called' **Name, Quarter Number, Quarter Name, Population**, and **Area** in hectares
>

In [902]:
url = 'https://en.wikipedia.org/wiki/Quarters_of_Paris'
data = requests.get(url)
df_list = pd.read_html(r.text) # this parses all the tables in webpages to a list
df = df_list[0]
df.head()

,Arrondissement(Districts),Quartiers(Quarters),Quartiers(Quarters).1,Population in1999[3],Area(hectares)[3],Map
0,"1st arrondissement(Called ""du Louvre"")",1st,Saint-Germain-l'Auxerrois,1672,86.9,NaN
1,"1st arrondissement(Called ""du Louvre"")",2nd,Les Halles,8984,41.2,NaN
2,"1st arrondissement(Called ""du Louvre"")",3rd,Palais-Royal,3195,27.4,NaN
3,"1st arrondissement(Called ""du Louvre"")",4th,Place-Vendôme,3044,26.9,NaN
4,"2nd arrondissement(Called ""de la Bourse"")",5th,Gaillon,1345,18.8,NaN


>BeautifulSoup could also be used with similar results, but is not necessary for this situation

>*We will need to also add two columns "Place" and "Arrondissement" in order to hold the split data from the first column "Arrondissement(Districts)"*

In [907]:
df['Arrond'] = df['Arrondissement(Districts)'].str.split('"').str[0]
df['Place'] = df['Arrondissement(Districts)'].str.split('"').str[1]
df
df.head()

,Arrondissement(Districts),Quartiers(Quarters),Quartiers(Quarters).1,Population in1999[3],Area(hectares)[3],Map,Arrond,Place
0,"1st arrondissement(Called ""du Louvre"")",1st,Saint-Germain-l'Auxerrois,1672,86.9,NaN,1st arrondissement(Called,du Louvre
1,"1st arrondissement(Called ""du Louvre"")",2nd,Les Halles,8984,41.2,NaN,1st arrondissement(Called,du Louvre
2,"1st arrondissement(Called ""du Louvre"")",3rd,Palais-Royal,3195,27.4,NaN,1st arrondissement(Called,du Louvre
3,"1st arrondissement(Called ""du Louvre"")",4th,Place-Vendôme,3044,26.9,NaN,1st arrondissement(Called,du Louvre
4,"2nd arrondissement(Called ""de la Bourse"")",5th,Gaillon,1345,18.8,NaN,2nd arrondissement(Called,de la Bourse


>From the list of Quarter names we can use geocoder to pull the cooordinates of the center of each area, </br>These are then returned in the "coords" table, this is then converted into the "df_coords" DataFrame, and the columns Latitude and Longitude are added to hold the data points.

In [ ]:
from geopy.geocoders import Bing
dframe = pd.DataFrame(df)
keycol = 'Quartiers(Quarters)'

def main(dframe, keycol):
    io = pd.DataFrame(dframe)
    def get_latitude(x):
      if x.latitude is None:
         x.latitude = None
      else:
         return x.latitude

    def get_longitude(x):
      if x.longitude is None:
         x.longitude = None
      else:
         return x.longitude
        
    geolocator = Bing('AuHXHPfLkAu40-qi_CPoyp42hlCZtdil0Lzwc8MSqC2sp2oxOvRXT_WAsJAN6NY7')
    geolocate_column = io[keycol].apply(geolocator.geocode)
    io['Latitude'] = geolocate_column.apply(get_latitude)
    io['Longitude'] = geolocate_column.apply(get_longitude)
    io.to_csv('geo{dframe}.csv')   
    dframe[keycol].apply(lambda x: geolocator.geocode(x).latitude if geolocator.geocode(x) != None else 'NF')
    
main(dframe, keycol)

In [ ]:
column_names = ['Number','Arrondissement', 'Address', 'City', 'Country','Point', 'Location', 'Latitude', 'Longitude', 'Altitude']
df_coords = pd.DataFrame(df, columns=['Latitude', 'Longitude'])
df_coords.head()              


>*Merge with "Arronds" and form into single DataFrame containing location information for Arrondissements in Paris.* </br> 

>"parcoord" is formed holding address and population information.  </br> 
Later the "parcoord_df" DataFrame is created 

>Column headers are defined below:</br>

* 'Number' ==> *The Number of the Arrondissment 1-20 *</br>
* 'Arr1' ==> *The abbrieviated common notation for the Arrondissement* </br>
* 'Arr2' ==> *The long abbrieviation for the Arrondissement* </br>
* 'Name' ==> *The central landmark or local name of the Arrondissement* </br>
* 'Area' ==> *Square miles of the Arrondissement* </br>
* 'Pop1999' ==> *population in 1999* </br>
* 'Pop2005' ==> *population in 2005* </br>
* 'PopPeak' ==> *the highest level the population had reached* </br>
* 'Density' ==> *population density of the area* </br>
* 'Mayor' ==> *current mayor of the Arrondissemnt* </br>
* 'Address' ==> *Address of the main street of the Arrondissment* </br>
* 'Latitude' ==> *Latitude portion of the coordinates of the city center* </br>
* 'Longitude' ==> *Longitude portion of the coordinate of the city center*


>*To look up the coordinates for each address in each of the rows in the DataFrame, we iterrate a function for 20 rows, and the Pandas DataFrame "arronds" is created, data is then wrangled and the table index is reset to display the information concisely and neatly.*

In [910]:
list = []
calc = 1

while int(calc) < 81:
    list.append(calc)
    calc = int(calc) + 1
print(list)

arronds = pd.DataFrame(df, columns=['Arrondissement', 'Number', 'City', 'Country', 'Latitude', 'Longitude', 'Postcode', 'Pop', 'Density'])
arronds['Arrondissement'] = df['Arrondissement(Districts)']
arronds['Arr'] = df['Quartiers(Quarters).1']
arronds['Latitude'] = df_coords['Latitude']
arronds['Longitude'] = df_coords['Longitude']
arronds['City'] = 'Paris'
arronds['Country'] = 'France'
arronds['Pop'] = df['Population in1999[3]']
arronds['Density']  = [arronds['Pop'].astype(float) / df['Area(hectares)[3]'].astype(float)]
arronds = arronds.set_index(80)
arronds.reset_index(drop=False, inplace=True)
arronds['Number'] = arrnum

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]


ValueError: Length of values does not match length of index

#### **Generate the Postal Codes for each Arrondissement**
Postal codes are given for each arrondissement within Paris where the numbers 750 proceed the number of the arrondissement, for example the 10th Arrondissement of Canal St. Martin woould have a postal zip of 75010.   

This is dohe utilizing basic ari


In [ ]:
arrnum = list[0:80]
post = pd.DataFrame(columns=['Post', 'Number'])
post['Number'] = arronds['Number'].astype(float)
post['Post'] = 75000
postcode = post['Post'] + post['Number']
arronds['Postcode'] = postcode.astype(str)
arronds.set_index(post['Number'], drop=True, inplace=False)
arronds['Postcode'] = arronds.Postcode.astype(str).str[0:-2]

arronds[['Number','Name']] = arronds.Arrond.str.split("(", expand=True)
arronds[['Name']] = arronds.Name.str.replace('Called "', '')
arronds[['Name']] = arronds.Name.str.replace('"', '')
arronds[['Name']] = arronds.Name.str.replace(')', '').str[3:]
arronds[['Number']] = arronds.Number.astype(str).str[0:-17]
arronds

In [ ]:
#qrts = pd.read_html('https://en.wikipedia.org/wiki/Quarters_of_Paris')
#for qrt in qrts:
#    print(qrt)

In [ ]:
/Quarters_of_Paris')
#for qrt in qrts:
#    print(qrt)#paris_qrts = pd.DataFrame(qrts[0])
#paris_qrts.head()

In [ ]:
#arrond_qrts = paris_qrts.merge(arronds, 'outer', left_on=None, right_on=None, left_index=False, right_index=False)

>*The coordinates generated seem to be inconsistant, so I will use another method to pull the numbers and then use the most accurate.*

#### **Scrape more detailed information**
>*Use Pandas html reader to pull additional website information from another source, using "arronds" as our list or "key"*

*Coerce into a Pandas DataFrame with the listed column headers, final data is displayed in the "paris_df" DataFrame*

In [ ]:
#column_names = ['Landmark', 'Names', 'Main Ave', 'Address', 'City', 'State', 'PostCode', 'Latitude', 'Longitude', 'Density', 'Area', 'Pop']
#paris_df = pd.DataFrame(df, columns=column_names)
#paris_df.reset_index(range(80), drop=True)
#paris_df['Landmark'] = df['Quartiers(Quarters)'],
#paris_df['Name'] = df['Arrondissement(Districts)'].strip('"'),
#paris_df['Main Ave'] = 'Nan',
#paris_df['Address']
#Arrondissement(Districts)	Quartiers(Quarters)	Quartiers(Quarters).1	Population in1999[3]	Area(hectares)[3]	Mapparis_df

In [751]:
#paris_df['Area (km2)'] = paris_df['Area (km2)'].replace(['16.305 km2 (6.295 sq mi)³7.846 km2 (3.029 sq mi)4'],'7.846 km2 (3.029 sq mi)')

In [752]:
#paris_df['Area (km2)'] = paris_df['Area (km2)'].replace(['16.305 km2 (6.295 sq mi)³7.846 km2 (3.029 sq mi)4'],'7.846 km2 (3.029 sq mi)')

In [753]:
#paris_df['Density (2017)(inhabitants per km2)'] = paris_df['Density (2017)(inhabitants per km2)'].replace(['8,657¹21,729²'],'21729')

In [754]:
#paris_df['Density (2017)(inhabitants per km2)'] = paris_df['Density (2017)(inhabitants per km2)'].replace(['8,472¹21,687²'],'21687')

In [759]:
parcoord = pd.DataFrame(arronds)
parcoord.reset_index(drop=True, inplace=False)
parcoord['Number'] = arronds['Number']
parcoord['Arr1'] = arronds['Number'].astype(str) +  'e'

parcoord['City'] = 'Paris'
parcoord['Country'] = 'France'
parcoord['Quarter'] = arronds['Arr']
address = parcoord['Name'].astype(str) + ', ' + parcoord['Arr'].astype(str) + ', ' + parcoord['Arr1'].astype(str) + ' Arrondissement, ' + parcoord['City'].astype(str) + ' ' + parcoord['Postcode'].astype(str)

parcoord['Postcode'] = postcode
parcoord['Address'] = address
parcoord['Latitude'] = arronds['Latitude']
parcoord['Longitude'] = arronds['Longitude']
parcoord['Density'] = paris_df['Density (2017)(inhabitants per km2)']
parcoord['Landmark'] = arronds['Arr1']

parcoord.drop
paris_df.rename(columns = {'Arrondissement (R for Right Bank, L for Left Bank)' : 'Arrondissement', 'Area (km)' : 'Area',  'Population(July 2017 estimate)' : 'Pop2017', 'Density (2017)(inhabitants per km2)' :'Density', 'Peak of population' : 'PeakPop'}, inplace=False)
parcoord.set_index(parcoord['Quarter'], drop=True, inplace=True, verify_integrity=True)
parcoord.head()

,index,Arrond,Number,City,Country,Latitude,Longitude,Postcode,Pop,Arr,Name,Arr1,Major Ave,Quarter,Address,Density,Landmark
Quarter,,,,,,,,,,,,,,,,,
Saint-Germain-l'Auxerrois,0,"1st arrondissement(Called ""du Louvre"")",1,Paris,France,30.198418,-93.968300,75001.0,1672,Saint-Germain-l'Auxerrois,Louvre,1e,"Louvre, Bourse, Temple, Hôtel-de-Ville",Saint-Germain-l'Auxerrois,"Louvre, Saint-Germain-l'Auxerrois, 1e Arrondis...",17924,1e
Les Halles,1,"1st arrondissement(Called ""du Louvre"")",1,Paris,France,27.113688,-82.417847,75002.0,8984,Les Halles,Louvre,1e,Panthéon,Les Halles,"Louvre, Les Halles, 1e Arrondissement, Paris 7...",23477,1e
Palais-Royal,2,"1st arrondissement(Called ""du Louvre"")",1,Paris,France,47.770084,-122.243752,75003.0,3195,Palais-Royal,Louvre,1e,Luxembourg,Palais-Royal,"Louvre, Palais-Royal, 1e Arrondissement, Paris...",19524,1e
Place-Vendôme,3,"1st arrondissement(Called ""du Louvre"")",1,Paris,France,38.733139,-82.995590,75004.0,3044,Place-Vendôme,Louvre,1e,Palais-Bourbon,Place-Vendôme,"Louvre, Place-Vendôme, 1e Arrondissement, Pari...",12761,1e
Gaillon,4,"2nd arrondissement(Called ""de la Bourse"")",2,Paris,France,29.898039,-94.328796,75005.0,1345,Gaillon,la Bourse,2e,Élysée,Gaillon,"la Bourse, Gaillon, 2e Arrondissement, Paris 7...",9631,2e


In [762]:
#columns_new = ['Arr1', 'Major Ave', 'Address', 'City', 'Country', 'Postcode', 'Latitude', 'Longitude', 'Density', 'Pop2017']
parcoord[parcoord.index.duplicated()]
parcoords = pd.DataFrame(parcoord[['Arr1','Latitude', 'Longitude', 'Postcode']])


#parcoords['Number'] = range(0,20)
#parcoords.set_index(['Arr1'], drop = True, inplace = True)

#parcoords = pd.DataFrame(parcoord, columns = columns_new)

#parcoords['Postcode'] = postcode
#dframe = parcoords
#keycol = 'Address'
#main(dframe, keycol)
parcoords['Latitude'].astype(float)
parcoords['Longitude'].astype(float)
parcoords['Postcode']
#parcoords.index.astype(float)
parcoords


,Arr1,Latitude,Longitude,Postcode
Quarter,,,,
Saint-Germain-l'Auxerrois,1e,30.198418,-93.968300,75001.0
Les Halles,1e,27.113688,-82.417847,75002.0
Palais-Royal,1e,47.770084,-122.243752,75003.0
Place-Vendôme,1e,38.733139,-82.995590,75004.0
Gaillon,2e,29.898039,-94.328796,75005.0
Vivienne,2e,36.509777,-119.561790,75006.0
Mail,2e,35.913177,-87.124908,75007.0
Bonne-Nouvelle,2e,37.999184,-121.343681,75008.0
Arts-et-Métiers,3e,29.903074,-90.001465,75009.0


*Create and push data into a new csv file*

In [810]:
parcoords.to_csv('paris.csv')
df_coords = np.array(parcoords[['Latitude', 'Longitude']])#.astype(float)

df_coords


array([[  30.19841766,  -93.96829987],
       [  27.11368752,  -82.41784668],
       [  47.77008438, -122.24375153],
       [  38.73313947,  -82.99559006],
       [  29.89803886,  -94.32879639],
       [  36.50977707, -119.56179047],
       [  35.91317749,  -87.12490845],
       [  37.99918365, -121.34368134],
       [  29.90307426,  -90.00146484],
       [  40.603154  ,  -79.7594365 ],
       [  40.10502555,  -85.68608835],
       [  39.91347301,  -80.73924099],
       [  40.07822065,  -93.61099529],
       [  40.75369995,  -74.0285786 ],
       [  40.1844101 , -105.13786316],
       [  35.33622237,  -78.68074916],
       [  41.55956287,  -89.10977732],
       [  41.85789871,  -87.6690979 ],
       [  29.3480302 ,  -99.1333359 ],
       [  36.14206735,  -78.75930864],
       [  42.0943625 ,  -80.13516851],
       [  41.33162508,  -80.40162319],
       [  45.538688  ,  -91.889809  ],
       [  32.83800906,  -85.18486844],
       [  39.22471511,  -85.80468415],
       [  37.7902155 , -1

In [811]:
locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('Champ de Mars, Paris, France')

print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

Latitude = 48.85614465, Longitude = 2.297820393322227


In [812]:
for column in map(df_coords[0], np.arange(80)[1]):
    df_coords[column] = df_coords[column].str.replace(',','.').astype(float)

TypeError: 'numpy.int64' object is not iterable

In [813]:
parcoords.info

<bound method DataFrame.info of                           Arr1   Latitude   Longitude  Postcode
Quarter                                                        
Saint-Germain-l'Auxerrois   1e  30.198418  -93.968300   75001.0
Les Halles                  1e  27.113688  -82.417847   75002.0
Palais-Royal                1e  47.770084 -122.243752   75003.0
Place-Vendôme               1e  38.733139  -82.995590   75004.0
Gaillon                     2e  29.898039  -94.328796   75005.0
Vivienne                    2e  36.509777 -119.561790   75006.0
Mail                        2e  35.913177  -87.124908   75007.0
Bonne-Nouvelle              2e  37.999184 -121.343681   75008.0
Arts-et-Métiers             3e  29.903074  -90.001465   75009.0
Enfants-Rouges              3e  40.603154  -79.759437   75010.0
Archives                    3e  40.105026  -85.686088   75011.0
Sainte-Avoye                3e  39.913473  -80.739241   75012.0
Saint-Merri                 4e  40.078221  -93.610995   75013.0
Saint-Ge

In [814]:
geo_data = 'arrondissements.geojson'

In [815]:
m = folium.Map([48.85614465, 2.297820393322227], zoom_start=11)

folium.GeoJson(geo_data).add_to(m)

m.save(os.path.join('GeoJson_paris.html'))

m

In [827]:
latitude = parcoords[['Latitude']]
longitude = parcoords['Longitude']
lat = latitude.values.tolist
lon = longitude.values.tolist

loc = df_coords
loc

array([[  30.19841766,  -93.96829987],
       [  27.11368752,  -82.41784668],
       [  47.77008438, -122.24375153],
       [  38.73313947,  -82.99559006],
       [  29.89803886,  -94.32879639],
       [  36.50977707, -119.56179047],
       [  35.91317749,  -87.12490845],
       [  37.99918365, -121.34368134],
       [  29.90307426,  -90.00146484],
       [  40.603154  ,  -79.7594365 ],
       [  40.10502555,  -85.68608835],
       [  39.91347301,  -80.73924099],
       [  40.07822065,  -93.61099529],
       [  40.75369995,  -74.0285786 ],
       [  40.1844101 , -105.13786316],
       [  35.33622237,  -78.68074916],
       [  41.55956287,  -89.10977732],
       [  41.85789871,  -87.6690979 ],
       [  29.3480302 ,  -99.1333359 ],
       [  36.14206735,  -78.75930864],
       [  42.0943625 ,  -80.13516851],
       [  41.33162508,  -80.40162319],
       [  45.538688  ,  -91.889809  ],
       [  32.83800906,  -85.18486844],
       [  39.22471511,  -85.80468415],
       [  37.7902155 , -1

In [828]:
CLIENT_ID = 'U4KD030Y4D1W4KHOW2TIKATHHUT4Z1NS2XR1T5FHYHTMCMPZ'
CLIENT_SECRET = 'AZUKDNM2EGIRD1PPR5DVQY114QXGWTAJUTN00Q5ABI0TBFF0' 
VERSION = '20180604'

LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: U4KD030Y4D1W4KHOW2TIKATHHUT4Z1NS2XR1T5FHYHTMCMPZ


In [831]:
parcoord

,index,Arrond,Number,City,Country,Latitude,Longitude,Postcode,Pop,Arr,Name,Arr1,Major Ave,Quarter,Address,Density,Landmark
Quarter,,,,,,,,,,,,,,,,,
Saint-Germain-l'Auxerrois,0,"1st arrondissement(Called ""du Louvre"")",1,Paris,France,30.198418,-93.968300,75001.0,1672,Saint-Germain-l'Auxerrois,Louvre,1e,"Louvre, Bourse, Temple, Hôtel-de-Ville",Saint-Germain-l'Auxerrois,"Louvre, Saint-Germain-l'Auxerrois, 1e Arrondis...",17924,1e
Les Halles,1,"1st arrondissement(Called ""du Louvre"")",1,Paris,France,27.113688,-82.417847,75002.0,8984,Les Halles,Louvre,1e,Panthéon,Les Halles,"Louvre, Les Halles, 1e Arrondissement, Paris 7...",23477,1e
Palais-Royal,2,"1st arrondissement(Called ""du Louvre"")",1,Paris,France,47.770084,-122.243752,75003.0,3195,Palais-Royal,Louvre,1e,Luxembourg,Palais-Royal,"Louvre, Palais-Royal, 1e Arrondissement, Paris...",19524,1e
Place-Vendôme,3,"1st arrondissement(Called ""du Louvre"")",1,Paris,France,38.733139,-82.995590,75004.0,3044,Place-Vendôme,Louvre,1e,Palais-Bourbon,Place-Vendôme,"Louvre, Place-Vendôme, 1e Arrondissement, Pari...",12761,1e
Gaillon,4,"2nd arrondissement(Called ""de la Bourse"")",2,Paris,France,29.898039,-94.328796,75005.0,1345,Gaillon,la Bourse,2e,Élysée,Gaillon,"la Bourse, Gaillon, 2e Arrondissement, Paris 7...",9631,2e
Vivienne,5,"2nd arrondissement(Called ""de la Bourse"")",2,Paris,France,36.509777,-119.561790,75006.0,2917,Vivienne,la Bourse,2e,Opéra,Vivienne,"la Bourse, Vivienne, 2e Arrondissement, Paris ...",27556,2e
Mail,6,"2nd arrondissement(Called ""de la Bourse"")",2,Paris,France,35.913177,-87.124908,75007.0,5783,Mail,la Bourse,2e,Entrepôt,Mail,"la Bourse, Mail, 2e Arrondissement, Paris 75007",31431,2e
Bonne-Nouvelle,7,"2nd arrondissement(Called ""de la Bourse"")",2,Paris,France,37.999184,-121.343681,75008.0,9595,Bonne-Nouvelle,la Bourse,2e,Popincourt,Bonne-Nouvelle,"la Bourse, Bonne-Nouvelle, 2e Arrondissement, ...",40183,2e
Arts-et-Métiers,8,"3rd arrondissement(Called ""du Temple"")",3,Paris,France,29.903074,-90.001465,75009.0,9560,Arts-et-Métiers,Temple,3e,Reuilly,Arts-et-Métiers,"Temple, Arts-et-Métiers, 3e Arrondissement, Pa...","8,657¹21,729²",3e


In [ ]:
parcoord['Density'] = parcoord['Density'].replace(['16.305 km2 (6.295 sq mi)³7.846 km2 (3.029 sq mi)4'],'7.846 km2 (3.029 sq mi)')

In [830]:
parcoord['Density'].astype(float)
from branca.colormap import linear

colormap = linear.YlGn_09.scale(
    parcoord['Density'].astype(float).min(),
    parcoord['Density'].astype(float).max())


ValueError: could not convert string to float: '9,169³19,0544'

In [ ]:
latitude = 48.85614465
longitude = 2.297820393322227

for index, row in df_coords.iterrows():
    paris_pop_map = folium.Map(location=[row['Latitude'], row['Longitude']], zoom_start=11, tiles='cartodbpositron')

folium.Choropleth(
    geo_data='arrondissements.geojson',
    columns=['Arr1', 'Density'],
    bins=20,
    location=location,
    fill_color='colormap',
    fill_opacity=0.3, 
    line_opacity=1.0,
    line_color='black',
    line_weight=2,
    label='Arr1',
    legend_name='Population by Arrondissement',
    radius=1000,
    threshold_scale=[0,1,2,3,4,5],  #[latitude, longitude],
    popup = ('Arrondissement: ' + str(address).capitalize() + '<br>')
).add_to(paris_pop_map)

paris_pop_map

In [ ]:
radius = 1000
LIMIT = 100
arrondissement = parcoord.Arr1
venues = []

for lat, lon, arrondissement in zip(parcoord['Latitude'], parcoord['Longitude'], parcoord['Arr1']): 
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lon,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    for venue in results:
        venues.append((
            arrondissement,
            lat, 
            lon, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
        
        

In [ ]:
venues_df = pd.DataFrame(venues)

venues_df.columns = ['Arrondissement', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
df_na = venues_df['Arrondissement'] != 'Na'
venues_df = venues_df[df_na]
print(venues_df.shape)
venues_df.head()
venues_df.tail()

In [ ]:
df = pd.DataFrame([{'c1':10, 'c2':100}, {'c1':11,'c2':110}, {'c1':12,'c2':120}])
df

In [ ]:
import geopandas

gdf = geopandas.read_file(geo_data)

m = folium.Map([48.85614465, 2.297820393322227], zoom_start=11)

folium.GeoJson(
    gdf, 
    style_function=lambda feature: {
        'fillColor': 'ffff00',
        'color': 'blue',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)

m.save(os.path.join('GeoJson_paris.html'))

m

In [ ]:
aveD = parcoord['Density'].astype(float).min() + parcoord['Density'].astype(float).max() / 2 
aveD

In [ ]:
id = parcoords['Density'].values.astype(np.int64)
def my_color_function(feature):
    if colormap[feature['id']] > aveD:
        return '#ff0000'
    else:
        return '#008000'


In [ ]:
for lat, lon, arronds in zip(parcoords['Latitude'], parcoords['Longitude'], parcoords['Arr1']): 
    paris_map = folium.Map(location=[lat, lon], zoom_start=13)

colordict = {0: 'lightblue', 1: 'lightgreen', 2: 'orange', 3: 'red', 4: 'beige'}

folium.Choropleth(
    geo_data = 'arrondissements.geojson',
    data = parcoords,
    columns = ['Number', 'Latitude', 'Longitude', 'Address', 'Name'],
    key_on = parcoords['Number'],
    bins = 20,
    fill_color = my_color_function('Density'),
    fill_opacity =  0.6,
    line_opacity = 1,
    name = None, 
    legend_name = '',
    overlay = True, 
    control = True, 
    topojson = None, 
    smooth_factor = None).add_to(paris_map),

if data is not None and not my_color_function(fill_color):
    raise ValueError('Please pass a valid color brewer code to '
    'fill_local. See docstring for valid codes.')
        
paris_map

# **Methodology** 

>*which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, 
>*if any, and what machine learnings were used and why.*


# **Discussion**

>*section where you discuss any observations you noted and any recommendations you can make based on the results.*# **Data Requirements and Descriptions**

# **Results**
>*Conclusion section where you conclude the report.*